In [1]:
import networkx as nx
# create a DAG
g = nx.DiGraph()
# add nl[j]s and edges to the DAG
g.add_edges_from([("A", "D"), ("B", "D"), ("B", "E"), ("B", "G"), ("C", "E"),("C", "F"), ("D", "I"), ("E", "G"),("E", "H"), ("F", "H"), ("G", "I"), ("G","J"), ("H", "J")])
nx.set_node_attributes(g, {"A": "PI", "B": "PI", "C":"PI", "D":"AND2", "E":"XOR2", "F":"INV", "G":"AND2", "H":"AND2", "I":"OR2", "J":"OR2"},name="gateType")
print(g.nodes(data=True))
# sort the nodes in topological order
nl = list(nx.topological_sort(g))
print('Nodes in topological order',nl)
# find children of a node
print('Successors of node B', list(g.successors("B")))
# find parents of a node
print('Parents of node H', list(g.predecessors("H")))
['C', 'F', 'A', 'B', 'D', 'E', 'G', 'H', 'I', 'J']

[('A', {'gateType': 'PI'}), ('D', {'gateType': 'AND2'}), ('B', {'gateType': 'PI'}), ('E', {'gateType': 'XOR2'}), ('G', {'gateType': 'AND2'}), ('C', {'gateType': 'PI'}), ('F', {'gateType': 'INV'}), ('I', {'gateType': 'OR2'}), ('H', {'gateType': 'AND2'}), ('J', {'gateType': 'OR2'})]
Nodes in topological order ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
Successors of node B ['D', 'E', 'G']
Parents of node H ['E', 'F']


['C', 'F', 'A', 'B', 'D', 'E', 'G', 'H', 'I', 'J']

# Getting the input from the netlist file

In [1]:
# Importing the necessary libraries
import networkx as nx

# Declaring variables for opening the files required
netwrok = "c432.net"
inputs = "c432.inputs"

# Defining a class to take in the vlaues from the network file
class Component:
	def __init__(self,name,g_type,input,output):
		self.name = name
		self.g_type = g_type
		self.input = input
		self.output = output
                
# Declaring a list to store the components
l = []

# Opening the file mentioned and storing it in the list
with open(netwrok) as f:
    lines = f.readlines()
    for line in lines:
        name,g_type,*input= line.split()
        output = line.split()[-1]
        Gate = Component(name,g_type,input,output)
        l.append(Gate)

# Declaring and creating the  edges list for the topoligal sorting
edges = []
g = nx.DiGraph()
for gate in l:
    if len(gate.input) == 3:
        edges.append((gate.input[0],gate.output))
        edges.append((gate.input[1],gate.output))
    else:
        edges.append((gate.input[0],gate.output))

# Making the diGraph for the topological sorting
g.add_edges_from(edges)

# Taking inputs from the inputs file and storing them in the values list as dictionaries
value = []
with open(inputs) as file:
    lineIn = file.readlines()
    pi = lineIn[0].split()
    for i in range(len(lineIn)-1):
        value.append({})
        for j in range(len(pi)):
            value[i].update({lineIn[0].split()[j]:lineIn[i+1].split()[j]})
value1 = value.copy()

# Saving the gate_types of eadh gate as a dictionary for later use in the gate function
gate_type = {}
for node in pi:
     gate_type.update({node:"PI"})
for node in l:
     gate_type.update({node.output:node.g_type})
nx.set_node_attributes(g,gate_type,name = "gateType")
print(gate_type)
# Making a topologically sorted list of the nodes
nl = list(nx.topological_sort(g))

print('Nodes in topological order',nl)




{'N1': 'PI', 'N4': 'PI', 'N8': 'PI', 'N11': 'PI', 'N14': 'PI', 'N17': 'PI', 'N21': 'PI', 'N24': 'PI', 'N27': 'PI', 'N30': 'PI', 'N34': 'PI', 'N37': 'PI', 'N40': 'PI', 'N43': 'PI', 'N47': 'PI', 'N50': 'PI', 'N53': 'PI', 'N56': 'PI', 'N60': 'PI', 'N63': 'PI', 'N66': 'PI', 'N69': 'PI', 'N73': 'PI', 'N76': 'PI', 'N79': 'PI', 'N82': 'PI', 'N86': 'PI', 'N89': 'PI', 'N92': 'PI', 'N95': 'PI', 'N99': 'PI', 'N102': 'PI', 'N105': 'PI', 'N108': 'PI', 'N112': 'PI', 'N115': 'PI', 'dummy_0': 'and2', 'N421': 'nor2', 'n_122': 'nor2', 'dummy_1': 'and2', 'dummy_2': 'and2', 'n_121': 'nand2', 'dummy_3': 'and2', 'dummy_4': 'and2', 'N431': 'nand2', 'dummy_5': 'and2', 'N432': 'nand2', 'n_116': 'inv', 'n_114': 'nand2', 'n_118': 'nand2', 'dummy_6': 'and2', 'N430': 'nand2', 'n_112': 'nand2', 'dummy_7': 'or2', 'n_110': 'nand2', 'n_107': 'nor2', 'n_111': 'inv', 'n_105': 'nand2', 'n_109': 'inv', 'dummy_8': 'and2', 'n_106': 'nand2', 'dummy_9': 'and2', 'n_117': 'nand2', 'dummy_10': 'and2', 'n_113': 'nand2', 'dummy_11

# Solving using the topological order

In [2]:
# Iterating through all the input sets given
for i in range(len(lineIn)-1):   

   # Iterating through the gates except pi
   for j in range(len(pi),len(nl)):

      # Finding the predecessors of the gate
      p = list(g.predecessors(nl[j]))
      out = 0 
      
      # Checking for all types of gates and performing the necessary operations
      if gate_type[nl[j]] == "nand2":
         a =  int(value[i][p[0]]) * int(value[i][p[1]])
         if a == 0 :
            out = 1
            
      if gate_type[nl[j]] == "and2":
         out =  int(value[i][p[0]]) * int(value[i][p[1]])

      if gate_type[nl[j]] == "nor2":
         if int(value[i][p[0]]) + int(value[i][p[1]]) < 1:
            out = 1
      
      if gate_type[nl[j]] == "or2":
         if int(value[i][p[0]]) + int(value[i][p[1]]) > 1:
            out = 1

      if gate_type[nl[j]] == "inv":
         if int(value[i][p[0]]) == 0:
            out = 1

      if gate_type[nl[j]] == "xnor2":
         if int(value[i][p[0]]) == int(value[i][p[1]]):
            out = 1

      if gate_type[nl[j]] == "or2":
         if int(value[i][p[0]]) != int(value[i][p[1]]):
            out = 1
      
      # Updating the output of the current gate into the value list
      value[i].update({nl[j]:out})
print(value[0])
           

{'N1': '0', 'N4': '0', 'N8': '1', 'N11': '0', 'N14': '1', 'N17': '1', 'N21': '0', 'N24': '0', 'N27': '0', 'N30': '0', 'N34': '1', 'N37': '1', 'N40': '1', 'N43': '0', 'N47': '0', 'N50': '0', 'N53': '0', 'N56': '1', 'N60': '1', 'N63': '0', 'N66': '1', 'N69': '0', 'N73': '1', 'N76': '1', 'N79': '1', 'N82': '0', 'N86': '1', 'N89': '1', 'N92': '1', 'N95': '1', 'N99': '0', 'N102': '0', 'N105': '0', 'N108': '1', 'N112': '0', 'N115': '0', 'n_2': 1, 'n_4': 0, 'n_8': 1, 'n_3': 0, 'n_1': 1, 'n_9': 0, 'n_5': 1, 'n_0': 0, 'n_6': 1, 'n_7': 1, 'n_20': 0, 'n_12': 1, 'n_10': 0, 'n_13': 1, 'n_14': 0, 'dummy_37': 1, 'n_15': 1, 'n_21': 0, 'n_19': 0, 'dummy_56': 1, 'n_11': 1, 'dummy_58': 1, 'dummy_57': 1, 'dummy_54': 0, 'n_17': 0, 'n_16': 0, 'dummy_55': 0, 'n_18': 1, 'n_22': 1, 'n_23': 0, 'n_24': 0, 'n_37': 1, 'n_39': 1, 'n_34': 1, 'n_27': 0, 'n_31': 1, 'n_43': 0, 'n_26': 1, 'n_29': 0, 'N223': 1, 'n_42': 0, 'n_38': 0, 'n_41': 0, 'n_40': 0, 'n_36': 0, 'n_35': 0, 'n_33': 1, 'n_28': 1, 'dummy_35': 1, 'n_32': 

# Using the Event - driven method

In [3]:
from queue import Queue
Event = Queue()
for item in nl[:len(pi)]:
    Event.put(item)
# for i in range(len(lineIn)-1):
    
i = 0
while not Event.empty():
    e = Event.get()
    for thing in list(g.successors(e)):
        if all(elem in value1[i] for elem in list(g.predecessors(thing))):
            Event.put(thing)

    if e not in value1[i]:
        print(e)
        p = list(g.predecessors(e))
        out = 0
        
        if gate_type[e] == "nand2":
            a =  int(value1[i][p[0]]) * int(value1[i][p[1]])
            if a == 0 :
                out = 1
                
        if gate_type[e] == "and2":
            out =  int(value1[i][p[0]]) * int(value1[i][p[1]])

        if gate_type[e] == "nor2":
            if int(value1[i][p[0]]) + int(value1[i][p[1]]) < 1:
                out = 1
        
        if gate_type[e] == "or2":
            if int(value1[i][p[0]]) + int(value1[i][p[1]]) > 1:
                out = 1

        if gate_type[e] == "inv":
            if int(value1[i][p[0]]) == 0:
                out = 1

        if gate_type[e] == "xnor2":
            if int(value1[i][p[0]]) == int(value1[i][p[1]]):
                out = 1

        if gate_type[e] == "or2":
            if int(value1[i][p[0]]) != int(value1[i][p[1]]):
                out = 1
                
        value1[i].update({e:out})

print(value1[0])

{'N1': '0', 'N4': '0', 'N8': '1', 'N11': '0', 'N14': '1', 'N17': '1', 'N21': '0', 'N24': '0', 'N27': '0', 'N30': '0', 'N34': '1', 'N37': '1', 'N40': '1', 'N43': '0', 'N47': '0', 'N50': '0', 'N53': '0', 'N56': '1', 'N60': '1', 'N63': '0', 'N66': '1', 'N69': '0', 'N73': '1', 'N76': '1', 'N79': '1', 'N82': '0', 'N86': '1', 'N89': '1', 'N92': '1', 'N95': '1', 'N99': '0', 'N102': '0', 'N105': '0', 'N108': '1', 'N112': '0', 'N115': '0', 'n_2': 1, 'n_4': 0, 'n_8': 1, 'n_3': 0, 'n_1': 1, 'n_9': 0, 'n_5': 1, 'n_0': 0, 'n_6': 1, 'n_7': 1, 'n_20': 0, 'n_12': 1, 'n_10': 0, 'n_13': 1, 'n_14': 0, 'dummy_37': 1, 'n_15': 1, 'n_21': 0, 'n_19': 0, 'dummy_56': 1, 'n_11': 1, 'dummy_58': 1, 'dummy_57': 1, 'dummy_54': 0, 'n_17': 0, 'n_16': 0, 'dummy_55': 0, 'n_18': 1, 'n_22': 1, 'n_23': 0, 'n_24': 0, 'n_37': 1, 'n_39': 1, 'n_34': 1, 'n_27': 0, 'n_31': 1, 'n_43': 0, 'n_26': 1, 'n_29': 0, 'N223': 1, 'n_42': 0, 'n_38': 0, 'n_41': 0, 'n_40': 0, 'n_36': 0, 'n_35': 0, 'n_33': 1, 'n_28': 1, 'dummy_35': 1, 'n_32': 

In [ ]:
for i in range(len(lineIn)-1):
    if set(value[i].items()) ^ set(value1[i].items()):
        print("There are differences")

In [ ]:
from collections import deque

# Define the list N
N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Define the size of the queue
n = 5

# Create a new deque object and add the first n values of N to the queue
queue = deque(N[:n])
l =[1,2,3]
queue.extend(l)
# Print the initial queue
print(queue)
if l in N:
    print("yes")